In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib
import matplotlib.pyplot as plt




In [3]:
df = pd.read_csv("dataset.csv",encoding='cp1252', index_col=0)

In [4]:
df.head()

,Mulheres,TempoM,Idade-M,Pais-M,Homens,TempoH,Idade-H,Pais-H
Ano,,,,,,,,
1991,Maria Luisa Servin,54:02:00,29,Mexico,Arturo Barrios,44:47:00,29,Mexicano
1992,Maria Del Carmen Diaz,54:00:00,32,Mexico,Simon Chemwoiywo,44:08:00,24,Quenia
1993,Hellen Kimaiyo,50:26:00,25,Quenia,Simon Chemwoiywo,43:20:00,25,Quenia
1994,Derartu Tulu,51:17:00,22,Etiopia,Ronaldo da Costa,44:11:00,24,Brasil
1995,Carmem Oliveira,50:53:00,30,Brasil,Paul Tergat,43:12:00,26,Quenia


In [5]:
print(range(1991))

range(0, 1991)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 1991 to 2018
Data columns (total 8 columns):
Mulheres    28 non-null object
TempoM      28 non-null object
Idade-M     28 non-null int64
Pais-M      28 non-null object
Homens      28 non-null object
TempoH      28 non-null object
Idade-H     28 non-null int64
Pais-H      28 non-null object
dtypes: int64(2), object(6)
memory usage: 2.0+ KB


In [7]:
df[['hour_TempoM', 'minute_TempoM', 'second_TempoM']] = df['TempoM'].astype(str).str.split(':', expand=True).astype(int)
df[['hour_TempoH', 'minute_TempoH', 'second_TempoH']] = df['TempoH'].astype(str).str.split(':', expand=True).astype(int)

In [8]:
df.drop(['TempoM','TempoH','second_TempoM','second_TempoH'], axis=1, inplace=True)
df= df.rename(columns={'hour_TempoM': 'minM', 'minute_TempoM': 'segM','hour_TempoH':'minH','minute_TempoH':'segH'} )
df.head()

,Mulheres,Idade-M,Pais-M,Homens,Idade-H,Pais-H,minM,segM,minH,segH
Ano,,,,,,,,,,
1991,Maria Luisa Servin,29,Mexico,Arturo Barrios,29,Mexicano,54,2,44,47
1992,Maria Del Carmen Diaz,32,Mexico,Simon Chemwoiywo,24,Quenia,54,0,44,8
1993,Hellen Kimaiyo,25,Quenia,Simon Chemwoiywo,25,Quenia,50,26,43,20
1994,Derartu Tulu,22,Etiopia,Ronaldo da Costa,24,Brasil,51,17,44,11
1995,Carmem Oliveira,30,Brasil,Paul Tergat,26,Quenia,50,53,43,12


In [9]:
df['SegM'] = df['minM']*60+df['segM']
df['SegH'] = df['minH']*60+df['segH']
df.drop(['minM','minH','segM','segH'], axis=1, inplace=True)

In [10]:
df['Dif'] = df.SegM - df.SegH

In [11]:
df

,Mulheres,Idade-M,Pais-M,Homens,Idade-H,Pais-H,SegM,SegH,Dif
Ano,,,,,,,,,
1991,Maria Luisa Servin,29,Mexico,Arturo Barrios,29,Mexicano,3242,2687,555
1992,Maria Del Carmen Diaz,32,Mexico,Simon Chemwoiywo,24,Quenia,3240,2648,592
1993,Hellen Kimaiyo,25,Quenia,Simon Chemwoiywo,25,Quenia,3026,2600,426
1994,Derartu Tulu,22,Etiopia,Ronaldo da Costa,24,Brasil,3077,2651,426
1995,Carmem Oliveira,30,Brasil,Paul Tergat,26,Quenia,3053,2592,461
1996,Roseli Machado,28,Brasil,Paul Tergat,27,Quenia,3152,2630,522
1997,Martha Tenorio,31,Equador,Emerson Iser Bem,24,Brasil,3124,2680,444
1998,Olivera Jevtic,21,Iugoslavia,Paul Tergat,29,Quenia,3095,2687,408
1999,Lydia Cheromei,22,Quenia,Paul Tergat,30,Quenia,3089,2675,414


In [12]:
df_Pais_H = df['Pais-H'].value_counts()
df_Pais_H.to_frame(name = 'Vitórias')


,Vitórias
Quenia,15
Brasil,6
Etiopia,6
Mexicano,1


In [13]:
df_H = df['Homens'].value_counts()
df_H.to_frame(name = 'Vitórias')

,Vitórias
Paul Tergat,5
Marilson Gomes dos Santos,3
Robert Kipkoech Cheruiyot,3
Dawit Admasu,2
James Kipsang,2
Simon Chemwoiywo,2
Edwin Kipsang,2
Emerson Iser Bem,1
Arturo Barrios,1
Belahy Bezabh,1


In [14]:
df_Pais_M = df['Pais-M'].value_counts()
df_Pais_M.to_frame(name = 'Vitórias')

,Vitórias
Quenia,14
Brasil,5
Etiopia,4
Mexico,2
Iugoslavia,1
Equador,1
Servia e Montenegro,1


In [15]:
df_M = df['Mulheres'].value_counts()
df_M.to_frame(name=' Vitórias')

,Vitórias
Lydia Cheromei,3
Yimer Wude Ayalew,3
Olivera Jevtic,2
Alice Timbilili,2
Jemima Sumgong,1
Maria Zeferina Baldaia,1
Margaret Okayo,1
Nancy Kipron,1
Carmem Oliveira,1
Marizete de Paula Rezende,1


In [19]:
df.head()

,Mulheres,Idade-M,Pais-M,Homens,Idade-H,Pais-H,SegM,SegH,Dif
Ano,,,,,,,,,
1991,Maria Luisa Servin,29,Mexico,Arturo Barrios,29,Mexicano,3242,2687,555
1992,Maria Del Carmen Diaz,32,Mexico,Simon Chemwoiywo,24,Quenia,3240,2648,592
1993,Hellen Kimaiyo,25,Quenia,Simon Chemwoiywo,25,Quenia,3026,2600,426
1994,Derartu Tulu,22,Etiopia,Ronaldo da Costa,24,Brasil,3077,2651,426
1995,Carmem Oliveira,30,Brasil,Paul Tergat,26,Quenia,3053,2592,461
